# Models Evaluator
Evaluate all the models on all the test datasets

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/luca-martinelli-09/orco-gan/blob/main/modelEvaluator.ipynb)

In [ ]:
# @markdown ## Setup project
# @markdown This section will download the datasets from GitHub to use for the training phase

if not os.path.exists("./datasets"):
    !git clone "https://github.com/luca-martinelli-09/orco-gan.git"

    %cd orco-gan/

In [ ]:
import os
import torch
from torch import nn
import pandas as pd
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
import math

import torchattacks
from torchattacks import PGD, FGSM

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("Torchattacks", torchattacks.__version__)

In [ ]:
# Detect if we have a GPU available
print("CUDA available:", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Set a manual seed

In [ ]:
SEED = 151836

def setSeed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

setSeed(SEED)

## Utils

In [ ]:
def printGPUStats():
    print('Using device:', device)
    print()

    # Additional Info when using cuda
    if device.type == 'cuda':
        print(torch.cuda.get_device_name(0))
        print('[💻 MEMORY USAGE]')
        print('[📌 ALLOCATED]', round(
            torch.cuda.memory_allocated(0) / 1024 ** 3, 1), 'GB')
        print('[🧮 CACHED]', round(
            torch.cuda.memory_reserved(0) / 1024 ** 3, 1), 'GB')


In [ ]:
def getSubDirs(dir):
    return [x for x in os.listdir(dir) if os.path.isdir(os.path.join(dir, x))]

In [ ]:
def getClassPercents(sizes):
    totalSize = np.sum(np.array(sizes))
    percents = []
    for size in sizes:
        percents.append(int(round((size / totalSize) * 100)))
    
    return percents

In [ ]:
def getBestScores(hist, key, min=False):
    scores = [x[key] for x in hist]

    if min:
        i = np.argmin(np.array(scores))
    else:
        i = np.argmax(np.array(scores))

    return hist[i], i

In [ ]:
def getMeanAndSDT(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

## Settings

In [ ]:
# @markdown ### Datasets
datasetsDir = "./datasets" # @param {type: "string"}

# @markdown ### Models
modelsDir = "./models" # @param {type: "string"}

inputSize = 224 # Specified for alexnet, resnet, vgg

## Models evaluations

In [ ]:
def getScores(labels, predicted):
    acc = torch.sum(predicted == labels) / len(predicted)

    tp = (labels * predicted).sum()
    tn = ((1 - labels) * (1 - predicted)).sum()
    fp = ((1 - labels) * predicted).sum()
    fn = (labels * (1 - predicted)).sum()

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    f1 = 2 * (precision * recall) / (precision + recall)

    return acc, precision, recall, f1

In [ ]:
def evaluateModel(model, dataloader, attack=None):
    model.eval()
    labelsOutputs = torch.tensor([]).to(device, non_blocking=True)
    labelsTargets = torch.tensor([]).to(device, non_blocking=True)

    setSeed(SEED)
    for inputs, labels in dataloader:
        inputs = attack(inputs, labels) if attack else inputs
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        labelsOutputs = torch.cat([labelsOutputs, preds], dim=0)
        labelsTargets = torch.cat([labelsTargets, labels], dim=0)
    
    acc, precision, recall, f1 = getScores(labelsTargets, labelsOutputs)

    return {
        "acc": acc.cpu().numpy(),
        "precision": precision.cpu().numpy(),
        "recall": recall.cpu().numpy(),
        "f1": f1.cpu().numpy()
    }


In [ ]:
class Normalize(nn.Module):
    def __init__(self, mean, std) :
        super(Normalize, self).__init__()
        self.register_buffer('mean', torch.Tensor(mean))
        self.register_buffer('std', torch.Tensor(std))
        
    def forward(self, input):
        # Broadcasting
        mean = self.mean.reshape(1, 3, 1, 1)
        std = self.std.reshape(1, 3, 1, 1)
        return (input - mean) / std

### Informations about models

In [ ]:
print("[🧠 MODELS INFORMATION]")

modelsInformation = []

for dataset in getSubDirs(modelsDir):
    print("\n" + "-" * 15)
    print("[🗃️ DATASET] {}".format(dataset))

    datasetDir = os.path.join(modelsDir, dataset)

    for modelType in getSubDirs(datasetDir):
        print("\n[🧮 MODEL TYPE] {}".format(modelType))

        modelsTypeDir = os.path.join(datasetDir, modelType)
        
        for model in os.listdir(modelsTypeDir):
            print("\n\t[🧠 MODEL] {}".format(model))

            path = os.path.join(modelsTypeDir, model)

            checkpoint = torch.load(path)
            
            bestScore, i = getBestScores(checkpoint["scores_history"], "f1")
            classBalancing = getClassPercents(checkpoint["dataset_sizes"])
            balancingStr = "/".join([str(x) for x in classBalancing])

            modelsInformation.append({
                "dataset": dataset,
                "model": checkpoint["model_name"],
                "epochs": len(checkpoint["scores_history"]),
                "balancing": balancingStr,
                "f-score": bestScore["f1"],
            })

            print("\tModel:", checkpoint["model_name"])
            print("\tEpochs:", len(checkpoint["scores_history"]))
            print("\tBalancing:", classBalancing)
            print("\tBest epoch:", i)
            print("\tBest F-Score:", bestScore["f1"])
            print("\tHistory:", [float(x["f1"]) for x in checkpoint["scores_history"]])

            torch.cuda.empty_cache()

modelsInformationDF = pd.DataFrame(modelsInformation)

printGPUStats()

In [ ]:
modelsInformationDF

### Evaluations

In [ ]:
filterAttacks = {
    "none": None
}

In [ ]:
from imageLimitedDataset import ImageLimitedDataset

print("[🧠 MODELS EVALUATION]")

modelsEvals = []

for dataset in getSubDirs(datasetsDir):
    print("\n" + "-" * 15)
    print("[🗃️ TEST DATASET] {}".format(dataset))
    
    datasetDir = os.path.join(datasetsDir, dataset)
    testDir = os.path.join(datasetDir, "test")

    for filterAttack in filterAttacks:
        print("[⚔️ FILTER ATTACK] {}".format(filterAttack))

        filterAttacker = filterAttacks[filterAttack]

        # Get the images and calculate mean and standard deviation
        toTensor = transforms.Compose([transforms.ToTensor()])
        testDataset = ImageLimitedDataset(
            testDir, transform=toTensor, use_cache=False, check_images=False, transform_filter=filterAttacker)

        for cls in testDataset.classes:
            cls_index = testDataset.class_to_idx[cls]
            num_cls = np.count_nonzero(
                np.array(testDataset.targets) == cls_index)
            print("\t[🧮 # ELEMENTS] {}: {}".format(cls, num_cls))

        setSeed(SEED)
        testDataLoader = DataLoader(testDataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)

        mean, std = getMeanAndSDT(testDataLoader)
        
        # Setup for normalization
        dataTransform = transforms.Compose([
            transforms.Resize(inputSize),
            transforms.ToTensor()
        ])

        torch.cuda.empty_cache()

        testDataset = ImageLimitedDataset(
            testDir, transform=dataTransform, use_cache=True, check_images=False, transform_filter=filterAttacker)
        
        setSeed(SEED)
        testDataLoader = DataLoader(
            testDataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)

        for root, _, fnames in sorted(os.walk(modelsDir, followlinks=True)):
            for fname in sorted(fnames):
                path = os.path.join(root, fname)
                model = torch.load(path)

                modelDataset = model["dataset"] if "dataset" in model.keys() else root.split(os.sep)[1]

                modelPercents = "/".join([str(x) for x in getClassPercents(model["dataset_sizes"])])

                print()
                print("[🧮 EVALUATING] {} - {} {}".format(
                    modelDataset,
                    model["model_name"],
                    modelPercents
                ))

                modelToTest = model["model"]

                # Add normalization
                modelToTest = nn.Sequential(
                    Normalize(mean, std),
                    modelToTest
                ).to(device, non_blocking=True)

                if not filterAttacker:
                    attacks = {
                        "none": None,
                        "FGSM": FGSM(modelToTest, eps=0.3)
                    }
                else:
                    attacks = {"none": None}

                for attack in attacks:
                    print("[⚔️ ATTACK] {}".format(attack))

                    attacker = attacks[attack]

                    scores = evaluateModel(modelToTest, testDataLoader, attacker)

                    modelsEvals.append({
                            "dataset": dataset,
                            "model": model["model_name"],
                            "attack": attack,
                            "filter": filterAttack,
                            "model_dataset": modelDataset,
                            "balancing": modelPercents,
                            "acc": scores["acc"],
                            "precision": scores["precision"],
                            "recall": scores["recall"],
                            "f1": scores["f1"],
                        })
                    
                    print("\tAcc: {:.4f}".format(scores["acc"]))
                    print("\tPre: {:.4f}".format(scores["precision"]))
                    print("\tRec: {:.4f}".format(scores["recall"]))
                    print("\tF-Score: {:.4f}".format(scores["f1"]))

                    torch.cuda.empty_cache()

In [ ]:
modelsEvalsDF = pd.DataFrame(modelsEvals)

In [ ]:
modelsEvalsDF

### Save evaluations

In [ ]:
modelsEvalsDF.to_csv("modelsEvaluations.csv")